# CRISPR screen analysis
# Genetics Bootcamp - Jan 9, 2023

This notebook will walk you through some basic analysis of example CRISPR screen data from human cells.

First, we must import some python libraries to manipulate and visualize the data. You will need pandas and bokeh installed in your jupyter environment.

In [ ]:
# import libraries
import operator
import numpy as np
import pandas as pd
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import output_notebook, show
from bokeh.transform import factor_cmap
output_notebook()

# Section 1: CRISPRi Growth Screen

The first dataset we will look at is a CRISPRi growth screen on K562 cells

Let's load the data from the screen

In [ ]:
# load and view the data
growthscreen_data = pd.read_csv("CRISPR_screendata/screendata_growth.csv", index_col=0, header=[0,1])

growthscreen_data

How many genes do we have data for?

This screen has two replicates. How well do the replicates correlate? What columns from the data table would be helpful in answering this question? Produce a plot below.

In [ ]:
#### choose two columns from the data above to plot against each other
# each column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
x_var = 
y_var = 


# the below code will produce a scatterplot of the columns selected above
source = ColumnDataSource(growthscreen_data)
x_var = '_'.join(x_var)
y_var = '_'.join(y_var)
graph=figure(x_axis_label=x_var, y_axis_label=y_var, 
             title="Growth Screen Replicate Correlation")
graph.scatter(source=source, x=x_var, y=y_var, fill_alpha=0.5)
graph.add_tools(HoverTool(tooltips=[('Gene','@gene')]))
show(graph)

Let's focus on the average of the two replicates. A volcano plot is helpful to understand the range of gene phenotype scores. Usually the -log10 transformed p-value is used as the y-axis variable. We will need to calucluate that transformation before we can build our graph.

In [ ]:
### choose a column from the data table above to transform and add this transformed column to the table
# column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
transform_column = 


# the code below will perform the transformation
growthscreen_data[(transform_column[0], "-log10 pvalue")] = growthscreen_data[transform_column].apply(np.log10) * -1

growthscreen_data

Now that we have transformed the p-values we can make a volcano plot

In [ ]:
### choose two columns from the data above to plot against each other
# each column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
x_var = 
y_var = 


# the below code will produce a scatterplot of the columns selected above
source = ColumnDataSource(growthscreen_data)
x_var = '_'.join(x_var)
y_var = '_'.join(y_var)
graph=figure(x_axis_label=x_var, y_axis_label=y_var, 
             title='Growth Screen Volcano Plot')
graph.scatter(source=source, x=x_var, y=y_var, fill_alpha=0.5)
graph.add_tools(HoverTool(tooltips=[('Gene','@gene')]))
show(graph)

Do most genes have a positive or negative phenotype score? Is this expected?

Before moving forward, we should do one more quality control check. The dataset contains a number of pseudogenes which serve as negative controls. Where do you expect these control genes to fall on our plot if the screen went well? Let's highlight these control genes.

In [ ]:
### choose two columns from the data above to plot against each other
### NOTE: we want to recreate the graph we just made above
# each column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
x_var = 
y_var = 


# this code adds a column to the table that indicates which genes are pseudogenes
pseudogene_data = growthscreen_data.copy()
pseudogene_data["NA", "pseudogene"] = pseudogene_data.index.str.startswith('pseudo', na=False)
pseudogene_data["NA", "pseudogene"] = pseudogene_data["NA", "pseudogene"].map({True: 'True', False: 'False'})
# the below code will produce a scatterplot of the columns selected above
index_cmap = factor_cmap("NA_pseudogene", palette=['blue', 'red'], factors=sorted(pseudogene_data["NA", "pseudogene"].unique()))
source = ColumnDataSource(pseudogene_data)
x_var = '_'.join(x_var)
y_var = '_'.join(y_var)
graph=figure(x_axis_label=x_var, y_axis_label=y_var,
             title='Growth Screen Volcano Plot')
graph.scatter(source=source, x=x_var, y=y_var, 
              fill_color=index_cmap, fill_alpha=0.5, 
              legend_field="NA_pseudogene")
graph.legend.title="pseudogene"
graph.add_tools(HoverTool(tooltips=[('Gene','@gene')]))
show(graph)

Thinking back to the plot as a whole, what are the genes that have the strongest phenotype scores? Is there a trend in which biological pathways these genes are associated with?

To answer this question you can hover over points to get the gene names. Alternatively, you can filter and sort the data table using the code below (advanced users with knowledge of pandas can modify the code below to filter the data on two or more columns at once).

In [ ]:
### choose a column to filter the data on
# column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
selection_column = 

### choose a value to set as a threshold
# should be a float
# e.g. 1.5
threshold_value = 

### choose genes greater or less than the threshold value
# should be a symbol enclosed in quotes
# e.g. "<" or ">"
greater_or_less_than = 


# the code below will produce a filtered and sorted data table
ops = {"<":operator.lt, ">":operator.gt}
op_func = ops[greater_or_less_than]
ascending_bool = greater_or_less_than == "<"
filtered_data = growthscreen_data.loc[
    op_func(growthscreen_data[selection_column], threshold_value)].sort_values(
    by=selection_column, ascending=ascending_bool)

filtered_data

You can google the gene names to get a sense of what they are. Alternatively, extract the gene names with the code below and copy/paste into the GO Enrichment Analysis web portal <geneontology.org>

In [ ]:
# the code below will extract the gene names from the filtered dataset above and put them into a txt file
# this makes it easy to copy and paste into GO
with open('growthscreen_filtered_genenames.txt', 'w') as outfile:
    genelist = list(filtered_data.index)
    for gene in genelist:
        outfile.write(gene+"\n")

One particularly interesting gene that does not fit this same category is GATA1. What does this gene do? Why do you think this gene has such a strong phenotype in this screen?

In [ ]:
# below is the data for the GATA1 gene
growthscreen_data.loc["GATA1"]

Well done! Pause here. The instructor will give you background on the next dataset we are going to look at.

# Section 2: CRISPRi Chemical Genetics Screen

Let's look at a more complicated chemical-genetics screen. In this screen CRISPRi expressing K562 cells were treated with rigosertib to determine what genes altered drug sensitivity and determine rigosertib's molecular mechanism.

In [ ]:
# load and view the data from the second screen
rigosertib_i_data = pd.read_csv("CRISPR_screendata/screendata_rigosertib_crispri.csv", index_col=0, header=[0,1,2])

rigosertib_i_data

This data table contains two different screens performed in parallel: a growth screen with no additional treatment (comparing t-start and t-end), and a rigosertib sensitivity screen (comparing + and - rigosertib treatment). Let's focus on the rigosertib sensitivity data to start.

In [ ]:
# this code will subset the table to only the rigosertib treatment
crispri_rigosertib = rigosertib_i_data.xs("rigosertib", level="selection", axis="columns")

crispri_rigosertib

Plot the replicates from the rigosertib screen to make sure they are well correlated.

In [ ]:
# choose two columns from the data above to plot against each other
# each column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
x_var = 
y_var = 


# the below code will produce a scatterplot of the columns selected above
source = ColumnDataSource(crispri_rigosertib)
x_var = '_'.join(x_var)
y_var = '_'.join(y_var)
graph=figure(x_axis_label=x_var, y_axis_label=y_var, 
             title="CRISPRi Rigosertib Sensitivity Replicate Correlation")
graph.scatter(source=source, x=x_var, y=y_var, fill_alpha=0.5)
graph.add_tools(HoverTool(tooltips=[('Gene','@gene')]))
show(graph)

Let's build a volcano plot similar to what we did before to see how the averaged phenotype scores are distributed.

Remember we need to start by -log10 transforming the p-value

In [ ]:
### choose a column from the data table above to transform and add this transformed column to the table
# column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
transform_column = 


# the code below will perform the transformation
crispri_rigosertib[(transform_column[0], "-log10 pvalue")] = crispri_rigosertib[transform_column].apply(np.log10) * -1

crispri_rigosertib

Now we can build the volcano plot.

In [ ]:
# choose two columns from the data above to plot against each other
# each column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
x_var = 
y_var = 


# the below code will produce a scatterplot of the columns selected above
source = ColumnDataSource(crispri_rigosertib)
x_var = '_'.join(x_var)
y_var = '_'.join(y_var)
graph=figure(x_axis_label=x_var, y_axis_label=y_var,
            title="CRISPRi Rigosertib Sensitivity Volcano Plot")
graph.scatter(source=source, x=x_var, y=y_var, fill_alpha=0.5)
graph.add_tools(HoverTool(tooltips=[('Gene','@gene')]))
show(graph)

Compare this plot to the volcano plot we made for the growth screen above. How do they differ? What could explain this difference?

As we did above, let's look at what genes have the strongest phenotype scores. Is there a pattern in the biological processes that these genes are involved in?

To answer this question you can hover over points to get the gene names. Alternatively, you can filter and sort the data table using the code below (advanced users with knowledge of pandas can modify the code below to filter the data on two or more columns at once).

In [ ]:
### choose a column to filter the data on
# column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
selection_column = 

### choose a value to set as a threshold
# should be a float
# e.g. 1.5
threshold_value = 

### choose genes greater or less than the threshold value
# should be a symbol enclosed in quotes
# e.g. "<" or ">"
greater_or_less_than = 


# the code below will produce a filtered and sorted data table
ops = {"<":operator.lt, ">":operator.gt}
op_func = ops[greater_or_less_than]
ascending_bool = greater_or_less_than == "<"
rigosertib_filtered_data = crispri_rigosertib.loc[
    op_func(crispri_rigosertib[selection_column], threshold_value)].sort_values(
    by=selection_column, ascending=ascending_bool)

rigosertib_filtered_data

You can google the gene names to get a sense of what they are. Alternatively, extract the gene names with the code below and copy/paste into the GO Enrichment Analysis web portal <geneontology.org>

In [ ]:
# the code below will extract the gene names from the filtered dataset above and put them into a txt file
# this makes it easy to copy and paste into GO
with open(datadir / 'crispri_rigosertib_filtered_genenames.txt', 'w') as outfile:
    genelist = list(rigosertib_filtered_data.index)
    for gene in genelist:
        outfile.write(gene+"\n")

To get a fuller idea of what is happening, let's pull in the other half of the dataset. Remember two different screens were performed in parallel: a growth screen with no additional treatment (comparing t-start and t-end), and a rigosertib sensitivity screen (comparing + and - rigosertib treatment). Maybe comparing the two screens will be informative.

Let's build a plot that compares the phenotypes in the parallel growth and sensitivity screens to see if the two halves of the dataset are correlated.

In [ ]:
# the code below will subset the data to relevant columns from the full dataset
parallel_data = rigosertib_i_data.xs("ave_Rep1_Rep2", level="replicate", axis="columns")

parallel_data

Now generate a plot using columns from this full dataset that will help us determine if the growth and sensitivity data are correlated.

In [ ]:
# choose two columns from the data above to plot against each other
# each column name should be a tuple of replicate and measurement
# e.g. ("growth", "transcripts")
x_var = 
y_var = 


# the below code will produce a scatterplot of the columns selected above
source = ColumnDataSource(parallel_data)
x_var = '_'.join(x_var)
y_var = '_'.join(y_var)
graph=figure(x_axis_label=x_var, y_axis_label=y_var,
            title="CRISPRi Growth and Rigosertib Sensitivity")
graph.scatter(source=source, x=x_var, y=y_var, fill_alpha=0.5)
graph.add_tools(HoverTool(tooltips=[('Gene','@gene')]))
show(graph)

What is effect on growth of knocking down genes with the strongest rigosertib sensitivity phenotypes? What does that tell us about most of the hits in the rigosertib sensitivity screen?

# Section 3: Combining CRISPRi and CRISPRa for Chemical Genetics

Let's also look at a third dataset. This comes from a CRISPRa screen of rigosertib sensitivity.

In [ ]:
# load and view the data from the CRISPRa screen
rigosertib_a_data = pd.read_csv("CRISPR_screendata/screendata_rigosertib_crispra.csv", index_col=0, header=[0,1,2])

rigosertib_a_data

Again, this data table contains two different screens performed in parallel: a growth screen with no additional treatment (comparing t-start and t-end), and a rigosertib sensitivity screen (comparing + and - rigosertib treatment). Let's focus on the rigosertib sensitivity data and make a volcano plot of the phenotype scores.

In [ ]:
# this code will subset the table to only the rigosertib treatment
crispra_rigosertib = rigosertib_a_data.xs("rigosertib", level="selection", axis="columns")

crispra_rigosertib

Remember we need to start by -log10 transforming the p-value

In [ ]:
### choose a column from the data table above to transform and add this transformed column to the table
# column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
transform_column = 


# the code below will perform the transformation
crispra_rigosertib[(transform_column[0], "-log10 pvalue")] = crispra_rigosertib[transform_column].apply(np.log10) * -1

crispra_rigosertib

Now we can build the volcano plot.

In [ ]:
# choose two columns from the data above to plot against each other
# each column name should be a tuple of replicate and measurement
# e.g. ("Rep1", "transcripts")
x_var = 
y_var = 


# the below code will produce a scatterplot of the columns selected above
source = ColumnDataSource(crispra_rigosertib)
x_var = '_'.join(x_var)
y_var = '_'.join(y_var)
graph=figure(x_axis_label=x_var, y_axis_label=y_var,
            title="CRISPRa Rigosertib Sensitivity Volcano Plot")
graph.scatter(source=source, x=x_var, y=y_var, fill_alpha=0.5)
graph.add_tools(HoverTool(tooltips=[('Gene','@gene')]))
show(graph)

How does this data compare to the other volcano plots we generated above?

The CRISPRi and CRISPRa datasets both interogate rigosertib sensitvity but with opposite perturbations. Combining the two datasets may lead to interesting observations.

In [ ]:
# this code will combine relevant columns from the two datasets into a single table
combined_data = pd.concat([
    crispri_rigosertib.xs("ave_Rep1_Rep2", level="replicate", axis="columns"), 
    crispra_rigosertib.xs("ave_Rep1_Rep2", level="replicate", axis="columns")], 
    axis=1, keys=["CRISPRi", "CRISPRa"])
combined_data.rename_axis(["screen", "measurement"], axis="columns", inplace=True)

combined_data

Let's visualize the relationship between both datasets.

In [ ]:
# choose two columns from the data above to plot against each other
# each column name should be a tuple of replicate and measurement
# e.g. ("CRISPRi", "transcripts")
x_var = 
y_var = 


# the below code will produce a scatterplot of the columns selected above
source = ColumnDataSource(combined_data)
x_var = '_'.join(x_var)
y_var = '_'.join(y_var)
graph=figure(x_axis_label=x_var, y_axis_label=y_var,
            title="CRISPRi & CRISPRa Rigosertib Sensitivity")
graph.scatter(source=source, x=x_var, y=y_var, fill_alpha=0.5)
graph.add_tools(HoverTool(tooltips=[('Gene','@gene')]))
show(graph)

Our overarching biological question is: "What is the molecular mechanism of rigosertib?". Where on the graph above would we potentially find genes that could be informative? What are those genes? Does this suggest a potential mechanism of action for rigosertib?